**Common functions and declarations**

**References:**
1.	https://www.sciencedirect.com/topics/computer-science/cosine-similarity#:~:text=Cosine%20similarity%20measures%20the%20similarity,in%20roughly%20the%20same%20direction.&text=Thus%2C%20each%20document%20is%20an,called%20a%20term%2Dfrequency%20vector.
2.	https://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python

In [56]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import csv
import io
import re
from scipy import spatial
import collections
import sqlite3
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import pandas as pd

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

# Open link and parse the content
def open_url(url):
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

def review_words(review):
    review2 = re.sub(r'[^\w\s]','',review.strip().lower())
    review2 = re.sub('[0-9]','',review2)
    text_token = word_tokenize(review2)
    review3 = [word for word in text_token if not word in stopwords.words()]
    
    return(review3) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejug\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## IMDB

In [32]:
# IMDB
imdb_url = 'https://www.imdb.com/chart/top/'
soup = open_url(imdb_url)

# Find all the movies
container = soup.find_all('td', class_ = 'titleColumn')

# Extract top 50 movies
movies = container[:50]
root = r'https://www.imdb.com/'
# imdb_m_dict is the dictionary with movie names and corresponding link
imdb_m_dict = {}
for movie in movies: 
    m_name = movie.find('a').text
    tag = movie.find('a')
    link = tag.get('href', None)
    imdb_m_dict[m_name] = root+link

revire_link = 'reviews?ref_=tt_ov_rt'
# Function to get movie reviews
def imdb_review():
    reviews_collection = []
    mv = {}
    for item in imdb_m_dict:
        l = imdb_m_dict[item]
        # Generate review link in x
        x = l[:38]+revire_link
    
        # Open url for all reviews and parse the content
        soup_review = open_url(x)
        
        r_container = soup_review.find_all('div', class_ = 'lister-item')
        
        # Get review for a movie and store in list m_reviews
        m_reviews = []
        for r in r_container:
            review1 = r.find('div', class_ = 'text').text
            review1 = re.sub('[^\w\s]','',review1)
            m_reviews.append(review1)
        tag = soup_review.find('div',class_='load-more-data')
        ajaxurl = tag.get('data-ajaxurl',None)
        datakey = tag.get('data-key',None)
        second_page = f'https://www.imdb.com/{ajaxurl}?paginationKey={datakey}'
        soup_review2 = open_url(second_page)
        r_container2 = soup_review2.find_all('div', class_ = 'lister-item')

        # Get review from page 2 for a movie and apeend the list m_reviews
        for r in r_container2:
            review2 = r.find('div', class_ = 'text').text
            review2 = re.sub('[^\w\s]','',review2)
            m_reviews.append(review2)

        mv[item] = m_reviews
        reviews_collection.append(m_reviews)
    # Return dictionary containing movie name and reviews
    return mv, reviews_collection    


imdb_d,imdb_allreviews = imdb_review()

In [34]:
# Write movie reviews to a csv file
data_imdb = pd.DataFrame.from_dict(imdb_d, orient='index')
data_imdb = data_imdb.transpose()
data_imdb.to_csv('Tejashri_reviews_imdb.csv')

In [71]:
# Count the number of words in reviews
imdb_kw = {}
# imdb_allreviews = imdb_allreviews[:1]
for r in imdb_allreviews:
    for i in r:
        rw = review_words(i)
        for w in rw:
            if w in imdb_kw:
                imdb_kw[w] = imdb_kw[w] + 1
            else:
                imdb_kw[w] = 1
print(imdb_kw)

{'shawshank': 95, 'redemption': 55, 'written': 6, 'directed': 2, 'frank': 12, 'darabont': 20, 'adaptation': 2, 'stephen': 12, 'king': 17, 'novella': 10, 'rita': 9, 'hayworth': 8, 'starring': 3, 'tim': 22, 'robbins': 35, 'morgan': 26, 'freeman': 37, 'film': 134, 'portrays': 4, 'story': 40, 'andy': 55, 'dufresne': 15, 'banker': 7, 'sentenced': 4, 'two': 18, 'life': 44, 'sentences': 2, 'state': 6, 'prison': 72, 'apparently': 2, 'murdering': 6, 'wife': 12, 'lover': 9, 'finds': 7, 'tough': 4, 'going': 21, 'solace': 3, 'friendship': 14, 'forms': 4, 'fellow': 5, 'inmate': 7, 'ellis': 5, 'red': 31, 'redding': 5, 'things': 19, 'start': 3, 'pick': 1, 'warden': 13, 'job': 2, 'befitting': 1, 'talents': 1, 'however': 14, 'arrival': 1, 'another': 8, 'vastly': 1, 'change': 3, 'themthere': 1, 'fanfare': 1, 'bunting': 1, 'put': 7, 'release': 3, 'back': 5, 'title': 11, 'didnt': 14, 'give': 6, 'much': 16, 'inkling': 1, 'anyone': 9, 'columbia': 1, 'pictures': 3, 'unsure': 1, 'market': 1, 'barely': 1, 'reg

In [72]:
imdb_sorted_kw = sorted(imdb_kw.items(), key=lambda x: x[1], reverse=True)
imdb_sorted_kw = imdb_sorted_kw[:50]
print(imdb_sorted_kw)

[('movie', 143), ('film', 134), ('shawshank', 95), ('prison', 72), ('best', 62), ('redemption', 55), ('andy', 55), ('hope', 52), ('time', 46), ('life', 44), ('ever', 42), ('like', 41), ('story', 40), ('freeman', 37), ('many', 37), ('robbins', 35), ('great', 32), ('seen', 32), ('movies', 32), ('see', 32), ('red', 31), ('people', 31), ('years', 29), ('dont', 27), ('morgan', 26), ('never', 26), ('good', 25), ('films', 25), ('first', 24), ('every', 23), ('really', 23), ('get', 23), ('acting', 23), ('way', 23), ('tim', 22), ('say', 22), ('well', 22), ('going', 21), ('makes', 21), ('could', 21), ('would', 21), ('darabont', 20), ('us', 20), ('feel', 20), ('made', 20), ('things', 19), ('watch', 19), ('times', 19), ('think', 19), ('two', 18)]


In [117]:
def imdb_get_genre():
    imdb_movie_genre = {}
    for item in imdb_m_dict:
        g = []
        # Open url for all movies and parse the content
        soup_genre = open_url(imdb_m_dict[item])
        g_container = soup_genre.find_all('div', class_ = 'subtext')
        for i in g_container:
            t = i.find_all('a')
            for n in t:
                review_url = n.get('href', None)
                if re.search('genres',review_url):
                    g.append(n.text.lower())
        imdb_movie_genre[item] = g
    return imdb_movie_genre
genre_imdb = imdb_get_genre()

In [124]:
# Processing the list of Genre to compute cosine similarity

im_vector = {}
for gen in genre_imdb:
    for gi in genre_imdb[gen]:
        if gi not in im_vector:
            im_vector[gi] = 1
        else:
            im_vector[gi] = im_vector[gi] + 1

od = sorted(im_vector.items(), key=lambda x: x[1], reverse=True)
od = dict(od)
print(od)

{'drama': 39, 'crime': 13, 'action': 13, 'adventure': 12, 'sci-fi': 6, 'comedy': 6, 'biography': 5, 'mystery': 5, 'thriller': 5, 'romance': 4, 'fantasy': 4, 'family': 3, 'war': 3, 'animation': 3, 'history': 2, 'western': 2, 'music': 2, 'horror': 1}


In [36]:
# Write from csv to SQL lite

conn = sqlite3.connect('MovieReviewDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieReviewTableIMDB')

df = pd.read_csv('Tejashri_reviews_imdb.csv')
df.to_sql('MovieReviewTableIMDB', conn, if_exists='append', index=False)
conn.commit()
conn.close()

## Metacritics

In [47]:
# Metacritics
meta_url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc'
soup = open_url(meta_url)
# Find all the movies
container = soup.find_all('td', class_ = 'clamp-summary-wrap')

# Extract top 50 movies
movies = container[:50]
root = r'https://www.metacritic.com'
# meta_m_dict is the dictionary with movie names and corresponding link
meta_m_dict = {}
for movie in movies: 
    m_name = movie.find('a',class_ = 'title').text
    tag = movie.find('a', class_ = 'title')
    link = tag.get('href', None)
    meta_m_dict[m_name] = root+link

# Function to get movie reviews
def meta_review():
    meta_allreviews = []
    mv = {}
    for item in meta_m_dict:
        movie_soup = open_url(meta_m_dict[item])
        movie_tag = movie_soup.find_all('a', class_ = 'see_all boxed oswald') 
        user_review_tag = movie_tag[1] 
        review_url = user_review_tag.get('href', None)
        if not review_url.endswith('user-reviews'):
            full_link = 'No user review yet.'
        else:
            full_link = 'https://www.metacritic.com'+ review_url
        if not full_link ==  'No user review yet.':
            # Open url for all reviews and parse the content
            soup_review = open_url(full_link)
            review_tag = soup_review.find_all('div', class_ = 'summary')
            top_50 = review_tag[:50]
            review_list = []
            for j in top_50:
                try:
                    review = j.select('span.blurb blurb_expanded')[0].text.strip()
                except:
                    review = j.select('div.review_body')[0].text.strip()
                review_list.append(review)
        else:
            review_list = 'nothing here'
        mv[item] = review_list
        meta_allreviews.append(review_list)
    # Return dictionary containing movie name and reviews
    return mv, meta_allreviews   

# Write movie reviews to a csv file
meta_d, meta_allreviews = meta_review()

In [49]:
# Write movie reviews to a csv file
data_meta = pd.DataFrame.from_dict(meta_d, orient='index')
data_meta = data_meta.transpose()
data_meta.to_csv('Tejashri_reviews_meta.csv')

In [74]:
# Count the number of words in reviews
meta_kw = {}
# meta_allreviews = meta_allreviews[:1]
for r in meta_allreviews:
    for i in r:
        rw = review_words(i)
        for w in rw:
            if w in meta_kw:
                meta_kw[w] = meta_kw[w] + 1
            else:
                meta_kw[w] = 1
    
# print(meta_kw)

In [75]:
meta_sorted_kw = sorted(meta_kw.items(), key=lambda x: x[1], reverse=True)
meta_sorted_kw = meta_sorted_kw[:50]
print(meta_sorted_kw)

[('film', 75), ('kane', 53), ('movie', 49), ('time', 29), ('citizen', 29), ('expand', 23), ('orson', 20), ('ever', 20), ('welles', 20), ('see', 18), ('made', 17), ('story', 17), ('watch', 16), ('best', 16), ('great', 15), ('cinema', 13), ('still', 13), ('love', 13), ('say', 13), ('greatest', 12), ('good', 11), ('movies', 11), ('character', 11), ('use', 11), ('like', 10), ('way', 9), ('films', 9), ('work', 9), ('life', 9), ('amazing', 9), ('kanes', 9), ('really', 9), ('many', 9), ('even', 8), ('camera', 8), ('tell', 8), ('cinematography', 8), ('performances', 8), ('must', 8), ('every', 8), ('first', 8), ('masterpiece', 7), ('shot', 7), ('history', 7), ('seen', 7), ('plot', 7), ('charles', 7), ('newspaper', 7), ('review', 7), ('present', 7)]


In [119]:
def meta_get_genre():
    meta_movie_genre = {}
    for item in meta_m_dict:
        # Open url for all movies and parse the content
        soup_genre = open_url(meta_m_dict[item])
        g_container = soup_genre.find('div', class_ = 'genres')
        t = g_container.text.strip().lower()
        y = re.sub(r'g.+:','',t)
        c = re.sub(',','',y)
        c = c.split()
        meta_movie_genre[item] = c
    return meta_movie_genre
genre_meta = meta_get_genre()
# print(genre_meta)

In [125]:
# Processing the list of Genre to compute cosine similarity
meta_vector = {}
for gen in genre_meta:
    for gi in genre_meta[gen]:
        if gi not in meta_vector:
            meta_vector[gi] = 1
        else:
            meta_vector[gi] = meta_vector[gi] + 1

m_vector = sorted(meta_vector.items(), key=lambda x: x[1], reverse=True)
m_vector = dict(m_vector)
print(m_vector)

{'drama': 39, 'thriller': 13, 'romance': 11, 'mystery': 10, 'comedy': 9, 'war': 5, 'history': 5, 'crime': 4, 'film-noir': 4, 'musical': 3, 'fantasy': 3, 'animation': 3, 'family': 3, 'adventure': 3, 'action': 3, 'horror': 3, 'western': 2, 'biography': 2, 'sport': 1, 'documentary': 1, 'music': 1}


In [51]:
# Write from csv to SQL lite

conn = sqlite3.connect('MovieReviewDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieReviewTableMetacritics')

df = pd.read_csv('Tejashri_reviews_meta.csv')
df.to_sql('MovieReviewTableMetacritics', conn, if_exists='append', index=False)
conn.commit()
conn.close()

## RottenTomatoes

In [107]:
# Rotten Tomatoes with critic reviews
rot_url = 'https://www.rottentomatoes.com/top/bestofrt/'
soup = open_url(rot_url)

# Find all the movies
container = soup.find_all('a',class_='unstyled articleLink')
m_container = []
m_names = []
for i in container:
    t = i.text
    tag = i.get('href', None)
    if re.search('/m/',tag):
        m_container.append(tag)
        m_names.append(i.text.strip())

# Extract top 50 movies
movies = m_container[:50]
root = r'https://www.rottentomatoes.com/'
# rot_m_dict is the dictionary with movie names and corresponding link
rot_m_dict = {}
count = 0
for movie in movies: 
    rot_m_dict[m_names[count]] = root+movie
    count += 1

# final_list = []
revire_link = '/reviews'
# Function to get movie reviews
def rot_review():
    rot_allreviews = []
    mv = {}
    for item in rot_m_dict:
        # Generate review links for all the pages containing the reviews 
        x1 = rot_m_dict[item]+revire_link
        x2 = rot_m_dict[item] + '/reviews?type=&sort=&page=2'
        x3 = rot_m_dict[item] + '/reviews?type=&sort=&page=3'

        # Open url1 for all reviews and parse the content
        soup_review = open_url(x1)
        r_container1 = soup_review.find_all('div',class_='review_desc')
        
        # Get review for a movie and store in list m_reviews
        m_reviews = []
        for c in r_container1:
            reviews1 = c.find('div',class_ = 'the_review').text.strip()
            reviews1 = re.sub('[^\w\s]','',reviews1)
            m_reviews.append(reviews1)
    
        # Open url2 for all reviews and parse the content
        soup_review2 = open_url(x2)
        r_container2 = soup_review2.find_all('div',class_='review_desc')

        for c in r_container2:
            reviews2 = c.find('div',class_ = 'the_review').text.strip()
            reviews2 = re.sub('[^\w\s]','',reviews2)
            m_reviews.append(reviews2)
            
        # Open url3 for all reviews and parse the content
        soup_review3 = open_url(x3)
        r_container3 = soup_review3.find_all('div',class_='review_desc')

        for c in r_container3:
            reviews3 = c.find('div',class_ = 'the_review').text.strip()
            reviews3 = re.sub('[^\w\s]','',reviews3)
            m_reviews.append(reviews3)
            
        m_reviews = m_reviews[:50]
        mv[item] = m_reviews
        rot_allreviews.append(m_reviews)

    # Return dictionary containing movie name and reviews
    return mv, rot_allreviews


r_mv, rot_allreviews = rot_review()

In [23]:
# Write reviews to csv
dat = pd.DataFrame.from_dict(r_mv, orient='index')
dat = dat.transpose()
dat.to_csv('Tejashri_reviews_rottenT.csv')

In [77]:
# Count the number of words in reviews
rot_kw = {}
# rot_allreviews = rot_allreviews[:1]
for r in rot_allreviews:
    for i in r:
        rw = review_words(i)
        for w in rw:
            if w in rot_kw:
                rot_kw[w] = rot_kw[w] + 1
            else:
                rot_kw[w] = 1
# print(rot_kw)

In [78]:
rot_sorted_kw = sorted(rot_kw.items(), key=lambda x: x[1], reverse=True)
rot_sorted_kw = rot_sorted_kw[:50]
print(rot_sorted_kw)

[('black', 21), ('panther', 17), ('movie', 12), ('superhero', 11), ('film', 9), ('marvel', 8), ('full', 6), ('best', 6), ('coogler', 5), ('review', 5), ('spanish', 5), ('important', 5), ('action', 4), ('ryan', 4), ('like', 4), ('story', 4), ('good', 4), ('movies', 4), ('design', 3), ('see', 3), ('marvels', 3), ('cultural', 3), ('cast', 3), ('diversity', 3), ('feels', 3), ('less', 3), ('time', 3), ('many', 3), ('films', 3), ('entire', 3), ('cooglers', 3), ('seen', 2), ('new', 2), ('king', 2), ('seem', 2), ('production', 2), ('ruth', 2), ('something', 2), ('special', 2), ('get', 2), ('likely', 2), ('date', 2), ('believe', 2), ('flick', 2), ('hollywood', 2), ('blockbuster', 2), ('set', 2), ('characters', 2), ('charisma', 2), ('work', 2)]


In [121]:
def rot_get_genre():
    rot_movie_genre = {}
    for item in rot_m_dict:
    
        # Open url for all movies and parse the content
        soup_genre = open_url(rot_m_dict[item])

        g_container = soup_genre.find_all('div', class_ = 'meta-value genre')

        for i in g_container:
            t = i.text
            t = re.sub('sci fi','sci-fi',t)
            t = re.sub('and','',t)
            t = re.sub(',','',t)
            n = t.split()
        rot_movie_genre[item] = n
    return rot_movie_genre
genre_rottenT = rot_get_genre()
# print(genre_rottenT)

In [126]:
# Processing the list of Genre to compute cosine similarity
rot_vector = {}
for gen in genre_rottenT:
    for gi in genre_rottenT[gen]:
        if gi not in rot_vector:
            rot_vector[gi] = 1
        else:
            rot_vector[gi] = rot_vector[gi] + 1

r_vector = sorted(rot_vector.items(), key=lambda x: x[1], reverse=True)
r_vector = dict(r_vector)
print(r_vector)

{'drama': 21, 'comedy': 18, 'adventure': 14, 'fantasy': 13, 'action': 10, 'kids': 8, 'family': 8, 'mystery': 7, 'thriller': 7, 'sci-fi': 5, 'animation': 5, 'crime': 5, 'horror': 4, 'romance': 4, 'musical': 2, 'music': 2, 'war': 2, 'history': 2}


In [31]:
# Write from csv to SQL lite

conn = sqlite3.connect('MovieReviewDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieReviewTableRot')

df = pd.read_csv('Tejashri_reviews_rottenT.csv')
df.to_sql('MovieReviewTableRot', conn, if_exists='append', index=False)
conn.commit()
conn.close()

In [128]:
set_genre = od.keys() | m_vector.keys() | r_vector.keys()

In [130]:
genre_total = {}
for i in set_genre:
    genre_total.update({i:0})
# print(genre_total)

{'sport': 0, 'family': 0, 'sci-fi': 0, 'drama': 0, 'fantasy': 0, 'music': 0, 'animation': 0, 'romance': 0, 'musical': 0, 'thriller': 0, 'biography': 0, 'documentary': 0, 'adventure': 0, 'film-noir': 0, 'mystery': 0, 'western': 0, 'action': 0, 'comedy': 0, 'history': 0, 'crime': 0, 'war': 0, 'kids': 0, 'horror': 0}


In [131]:
# imdb
genre_imdb = genre_total.copy()
for i in list(od.keys()):
    if i in list(od.keys()):
        genre_imdb[i] = od[i]
print(genre_imdb)

{'sport': 0, 'family': 3, 'sci-fi': 6, 'drama': 39, 'fantasy': 4, 'music': 2, 'animation': 3, 'romance': 4, 'musical': 0, 'thriller': 5, 'biography': 5, 'documentary': 0, 'adventure': 12, 'film-noir': 0, 'mystery': 5, 'western': 2, 'action': 13, 'comedy': 6, 'history': 2, 'crime': 13, 'war': 3, 'kids': 0, 'horror': 1}


In [132]:
# meta
genre_meta = genre_total.copy()
for i in list(m_vector.keys()):
    if i in list(m_vector.keys()):
        genre_meta[i] = m_vector[i]
print(genre_meta)

{'sport': 1, 'family': 3, 'sci-fi': 0, 'drama': 39, 'fantasy': 3, 'music': 1, 'animation': 3, 'romance': 11, 'musical': 3, 'thriller': 13, 'biography': 2, 'documentary': 1, 'adventure': 3, 'film-noir': 4, 'mystery': 10, 'western': 2, 'action': 3, 'comedy': 9, 'history': 5, 'crime': 4, 'war': 5, 'kids': 0, 'horror': 3}


In [133]:
# rot
genre_rot = genre_total.copy()
for i in list(r_vector.keys()):
    if i in list(r_vector.keys()):
        genre_rot[i] = r_vector[i]
print(genre_rot)

{'sport': 0, 'family': 8, 'sci-fi': 5, 'drama': 21, 'fantasy': 13, 'music': 2, 'animation': 5, 'romance': 4, 'musical': 2, 'thriller': 7, 'biography': 0, 'documentary': 0, 'adventure': 14, 'film-noir': 0, 'mystery': 7, 'western': 0, 'action': 10, 'comedy': 18, 'history': 2, 'crime': 5, 'war': 2, 'kids': 8, 'horror': 4}


In [134]:
# Find the vectors to compute cosine similarity
print('imdb')
iv = list(genre_imdb.values())
print(iv)
print('meta')
mv = list(genre_meta.values())
print(mv)
print('rotten')
rv = list(genre_rot.values())
print(rv)

imdb
[0, 3, 6, 39, 4, 2, 3, 4, 0, 5, 5, 0, 12, 0, 5, 2, 13, 6, 2, 13, 3, 0, 1]
meta
[1, 3, 0, 39, 3, 1, 3, 11, 3, 13, 2, 1, 3, 4, 10, 2, 3, 9, 5, 4, 5, 0, 3]
rotten
[0, 8, 5, 21, 13, 2, 5, 4, 2, 7, 0, 0, 14, 0, 7, 0, 10, 18, 2, 5, 2, 8, 4]


In [135]:
# Find cosine Similarity
cs_imdb_meta = 1 - spatial.distance.cosine(iv, mv)
cs_imdb_rot = 1 - spatial.distance.cosine(iv, rv)
cs_meta_rot = 1 - spatial.distance.cosine(mv, rv)

In [136]:
print('Cosine similarity between genres IMDB and Metacritics: ',cs_imdb_meta)
print('Cosine similarity between genres IMDB and RottenTomatoes: ',cs_imdb_rot)
print('Cosine similarity between genres Metacritics and RottenTomatoes: ',cs_meta_rot)

Cosine similarity between genres IMDB and Metacritics:  0.8857105210415298
Cosine similarity between genres IMDB and RottenTomatoes:  0.80881563991013
Cosine similarity between genres Metacritics and RottenTomatoes:  0.7580139789525701


In [81]:
# Cosine similarity for the movie reviews
# Convert the sorted lists to dictionaries
imdb_sorted_kw = dict(imdb_sorted_kw)
meta_sorted_kw = dict(meta_sorted_kw)
rot_sorted_kw = dict(rot_sorted_kw)

In [83]:
# set_total = 
set_total = imdb_sorted_kw.keys() | meta_sorted_kw.keys() | rot_sorted_kw.keys()

In [110]:
dict_total = {}
for i in set_total:
    dict_total.update({i:0})
# print(dict_total)

In [137]:
# imdb
dict_imdb = dict_total.copy()
for i in list(dict_imdb.keys()):
    if i in list(imdb_sorted_kw.keys()):
        dict_imdb[i] = imdb_sorted_kw[i]
print(dict_imdb)
# print(dict_total)

{'new': 0, 'cultural': 0, 'every': 23, 'less': 0, 'good': 25, 'robbins': 35, 'really': 23, 'production': 0, 'panther': 0, 'story': 40, 'first': 24, 'redemption': 55, 'shot': 0, 'marvels': 0, 'masterpiece': 0, 'freeman': 37, 'cast': 0, 'prison': 72, 'kanes': 0, 'even': 0, 'black': 0, 'cooglers': 0, 'great': 32, 'welles': 0, 'date': 0, 'citizen': 0, 'love': 0, 'characters': 0, 'tim': 22, 'ever': 42, 'expand': 0, 'design': 0, 'flick': 0, 'cinematography': 0, 'life': 44, 'movie': 143, 'amazing': 0, 'something': 0, 'never': 26, 'blockbuster': 0, 'ruth': 0, 'get': 23, 'seem': 0, 'shawshank': 95, 'present': 0, 'say': 22, 'darabont': 20, 'superhero': 0, 'time': 46, 'set': 0, 'made': 20, 'morgan': 26, 'diversity': 0, 'feel': 20, 'special': 0, 'hope': 52, 'work': 0, 'still': 0, 'tell': 0, 'times': 19, 'camera': 0, 'king': 0, 'like': 41, 'kane': 0, 'charles': 0, 'charisma': 0, 'see': 32, 'years': 29, 'film': 134, 'way': 23, 'us': 20, 'many': 37, 'use': 0, 'full': 0, 'hollywood': 0, 'entire': 0, '

In [138]:
# meta
dict_meta = dict_total.copy()
for i in list(dict_meta.keys()):
    if i in list(meta_sorted_kw.keys()):
        dict_meta[i] = meta_sorted_kw[i]
print(dict_meta)

{'new': 0, 'cultural': 0, 'every': 8, 'less': 0, 'good': 11, 'robbins': 0, 'really': 9, 'production': 0, 'panther': 0, 'story': 17, 'first': 8, 'redemption': 0, 'shot': 7, 'marvels': 0, 'masterpiece': 7, 'freeman': 0, 'cast': 0, 'prison': 0, 'kanes': 9, 'even': 8, 'black': 0, 'cooglers': 0, 'great': 15, 'welles': 20, 'date': 0, 'citizen': 29, 'love': 13, 'characters': 0, 'tim': 0, 'ever': 20, 'expand': 23, 'design': 0, 'flick': 0, 'cinematography': 8, 'life': 9, 'movie': 49, 'amazing': 9, 'something': 0, 'never': 0, 'blockbuster': 0, 'ruth': 0, 'get': 0, 'seem': 0, 'shawshank': 0, 'present': 7, 'say': 13, 'darabont': 0, 'superhero': 0, 'time': 29, 'set': 0, 'made': 17, 'morgan': 0, 'diversity': 0, 'feel': 0, 'special': 0, 'hope': 0, 'work': 9, 'still': 13, 'tell': 8, 'times': 0, 'camera': 8, 'king': 0, 'like': 10, 'kane': 53, 'charles': 7, 'charisma': 0, 'see': 18, 'years': 0, 'film': 75, 'way': 9, 'us': 0, 'many': 9, 'use': 11, 'full': 0, 'hollywood': 0, 'entire': 0, 'ryan': 0, 'perfo

In [139]:
# rot
dict_rot = dict_total.copy()
for i in list(dict_meta.keys()):
    if i in list(rot_sorted_kw.keys()):
        dict_rot[i] = rot_sorted_kw[i]
print(dict_rot)

{'new': 2, 'cultural': 3, 'every': 0, 'less': 3, 'good': 4, 'robbins': 0, 'really': 0, 'production': 2, 'panther': 17, 'story': 4, 'first': 0, 'redemption': 0, 'shot': 0, 'marvels': 3, 'masterpiece': 0, 'freeman': 0, 'cast': 3, 'prison': 0, 'kanes': 0, 'even': 0, 'black': 21, 'cooglers': 3, 'great': 0, 'welles': 0, 'date': 2, 'citizen': 0, 'love': 0, 'characters': 2, 'tim': 0, 'ever': 0, 'expand': 0, 'design': 3, 'flick': 2, 'cinematography': 0, 'life': 0, 'movie': 12, 'amazing': 0, 'something': 2, 'never': 0, 'blockbuster': 2, 'ruth': 2, 'get': 2, 'seem': 2, 'shawshank': 0, 'present': 0, 'say': 0, 'darabont': 0, 'superhero': 11, 'time': 3, 'set': 2, 'made': 0, 'morgan': 0, 'diversity': 3, 'feel': 0, 'special': 2, 'hope': 0, 'work': 2, 'still': 0, 'tell': 0, 'times': 0, 'camera': 0, 'king': 2, 'like': 4, 'kane': 0, 'charles': 0, 'charisma': 2, 'see': 3, 'years': 0, 'film': 9, 'way': 0, 'us': 0, 'many': 3, 'use': 0, 'full': 6, 'hollywood': 2, 'entire': 3, 'ryan': 4, 'performances': 0, '

In [114]:
# Get word frequency vectors
imdb_wv = list(dict_imdb.values())
print(imdb_wv)
meta_wv = list(dict_meta.values())
print(meta_wv)
rot_wv = list(dict_rot.values())
print(rot_wv)

[0, 0, 23, 0, 25, 35, 23, 0, 0, 40, 24, 55, 0, 0, 0, 37, 0, 72, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 22, 42, 0, 0, 0, 0, 44, 143, 0, 0, 26, 0, 0, 23, 0, 95, 0, 22, 20, 0, 46, 0, 20, 26, 0, 20, 0, 52, 0, 0, 0, 19, 0, 0, 41, 0, 0, 0, 32, 29, 134, 23, 20, 37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 32, 0, 0, 55, 23, 21, 27, 22, 32, 0, 0, 0, 19, 0, 21, 0, 25, 31, 0, 62, 0, 18, 21, 31, 0, 0, 19, 21, 0, 0]
[0, 0, 8, 0, 11, 0, 9, 0, 0, 17, 8, 0, 7, 0, 7, 0, 0, 0, 9, 8, 0, 0, 15, 20, 0, 29, 13, 0, 0, 20, 23, 0, 0, 8, 9, 49, 9, 0, 0, 0, 0, 0, 0, 0, 7, 13, 0, 0, 29, 0, 17, 0, 0, 0, 0, 0, 9, 13, 8, 0, 8, 0, 10, 53, 7, 0, 18, 0, 75, 9, 0, 9, 11, 0, 0, 0, 0, 8, 7, 0, 8, 0, 0, 11, 20, 7, 0, 0, 0, 0, 0, 7, 0, 11, 12, 0, 0, 0, 0, 9, 0, 7, 16, 13, 0, 0, 0, 0, 0, 16, 0, 0, 7]
[2, 3, 0, 3, 4, 0, 0, 2, 17, 4, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 21, 3, 0, 0, 2, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 12, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 11, 3, 2, 0, 0, 3, 0, 2, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0, 2, 3, 0, 9, 0, 0, 3, 0, 6, 2, 3, 4, 0, 0, 3

In [115]:
# Find cosine similarity
cosine_word_imdb_meta = 1 - spatial.distance.cosine(imdb_wv, meta_wv)
cosine_word_imdb_rot = 1 - spatial.distance.cosine(imdb_wv, rot_wv)
cosine_word_meta_rot = 1 - spatial.distance.cosine(meta_wv, rot_wv)
print('Similarity score between reviews IMDB and Metacrirics',cosine_word_imdb_meta)
print('Similarity score between reviews IMDB and Rotten Tomatoes',cosine_word_imdb_rot)
print('Similarity score between reviews Rotten Tomatoes and Metacrirics',cosine_word_meta_rot)

Similarity score between reviews IMDB and Metacrirics 0.6010067032553557
Similarity score between reviews IMDB and Rotten Tomatoes 0.3488813856404567
Similarity score between reviews Rotten Tomatoes and Metacrirics 0.33040246776197557
